In [7]:
import polars as pl
import statsmodels.formula.api as smf
import numpy as np
from stargazer.stargazer import Stargazer
from linearmodels.panel import PanelOLS

In [11]:
df = pl.read_parquet("../data/combined_all.parquet")
pd_df = df.to_pandas().set_index(["country", "year"])

In [12]:
pd_df["arwu_ranked_num_pc"] = pd_df["arwu_ranked_num"] / pd_df["population"] * 1_000_000
pd_df["gdp_pc"] = np.log(pd_df["gdp_pc"])
pd_df["population"] = np.log(pd_df["population"])

In [18]:
group1 = pd_df[(pd_df["arwu_ranked_num"].notnull()) & (pd_df["math"].notnull()) & (pd_df["imo_total_score"].notnull()) & (pd_df["primary_completion"].notnull()) & (pd_df["lower_sec_completion"].notnull()) & (pd_df["upper_sec_completion"].notnull()) & (pd_df["democracy_eiu"].notnull())]
group1.shape

(109, 15)

In [57]:
group2 = pd_df[(pd_df["gdp_pc"].notnull()) & (pd_df["arwu_ranked_num"].notnull()) & (pd_df["imo_total_score"].notnull()) & (pd_df["primary_completion"].notnull()) & (pd_df["lower_sec_completion"].notnull()) & (pd_df["upper_sec_completion"].notnull()) & (pd_df["democracy_eiu"].notnull())]
group2.shape

(746, 15)

In [84]:
pisa_time_fe = PanelOLS.from_formula("gdp_pc_growth ~ gdp_pc + imo_total_score + math + in_math99 + arwu_ranked_num*gdp_pc + primary_completion + lower_sec_completion + upper_sec_completion + population + democracy_eiu + TimeEffects", group1).fit()

In [85]:
pisa_time_country_fe = PanelOLS.from_formula("gdp_pc_growth ~ gdp_pc + imo_total_score + math + in_math99 + arwu_ranked_num*gdp_pc + primary_completion + lower_sec_completion + upper_sec_completion + population + democracy_eiu + TimeEffects + EntityEffects", group1).fit()

In [86]:
pisa_no_math_time_fe = PanelOLS.from_formula("gdp_pc_growth ~ gdp_pc + imo_total_score + in_math99 + arwu_ranked_num*gdp_pc + primary_completion + lower_sec_completion + upper_sec_completion + population + democracy_eiu + TimeEffects", group1).fit()

In [87]:
non_pisa = PanelOLS.from_formula("gdp_pc_growth ~ gdp_pc + democracy_eiu + imo_total_score + arwu_ranked_num*gdp_pc + primary_completion + lower_sec_completion + upper_sec_completion + population + TimeEffects", group2).fit()

In [88]:
non_pisa_no_interac = PanelOLS.from_formula("gdp_pc_growth ~ gdp_pc + democracy_eiu + imo_total_score + arwu_ranked_num + primary_completion + lower_sec_completion + upper_sec_completion + population + TimeEffects", group2).fit()

In [95]:
non_pisa

Dep. Variable:,gdp_pc_growth,R-squared:,0.0974
Estimator:,PanelOLS,R-squared (Between):,-9.5763
No. Observations:,746,R-squared (Within):,0.0035
Date:,"Sun, Mar 31 2024",R-squared (Overall):,-5.6155
Time:,17:23:35,Log-likelihood,-1833.6
Cov. Estimator:,Unadjusted,,
,,F-statistic:,8.6589
Entities:,137,P-value,0.0000
Avg Obs:,5.4453,Distribution:,"F(9,722)"
Min Obs:,1.0000,,
Max Obs:,15.000,F-statistic (robust):,8.6589


In [96]:
pisa

Dep. Variable:,gdp_pc_growth,R-squared:,0.2814
Estimator:,PanelOLS,R-squared (Between):,-49.950
No. Observations:,109,R-squared (Within):,0.0539
Date:,"Sun, Mar 31 2024",R-squared (Overall):,-39.273
Time:,17:15:24,Log-likelihood,-220.43
Cov. Estimator:,Unadjusted,,
,,F-statistic:,3.3461
Entities:,48,P-value,0.0006
Avg Obs:,2.2708,Distribution:,"F(11,94)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,3.3461
